# Prompting Examples

Material is mainly coming from the part of short course designed by DeepLearning.ai, offered as [ChatGPT Prompt Engineering for Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

Some prompt and text examples are modified with different ingridients!

## Setup

If you are interested in doing experiments via python, you need to have OpenAI API key for this (not the only choice but material is designed relying on this tool)

#### Load the API key and relevant Python libaries.

In [1]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 579.0 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [5]:
import openai
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

# os.environ["OPENAI_API_KEY"] = "Your OpenAI-API-key"

#### Main Helper function
Throughout this script, you can use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs.  



In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note:** This and all other lab notebooks of this course use OpenAI library version `0.27.0`.

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the `get_completion` function:

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

### Main Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

- We are using a backslash \ to make the text fit on the screen without inserting newline '\n' characters.

In [7]:
text = f"""
With the recent advancements in the field of Natural Language Processing (NLP), \
Generative AI (Gen-AI) tools are rapidly evolving and have already become powerful tools \
for various sectors. Including the widely known tool ChatGPT, there are various competitors \
in the market with recently proposed open-source alternatives. Regarding the method under the hood \
and the data set they are trained on, different tools have certain capabilities and limitations.\
\To have a better conservation with these Gen-AI tools and fully harness their potential, \
it is crucial to be aware of the concept of prompt engineering and power. \
This training workshop aims to introduce the main strategies of prompt engineering including \
tips and certain practices to achieve optimal outputs with Gen-AI tools. After providing a hands-on \
overview of prompt engineering, certain features of ChatGPT will be discussed for fostering \
the research practices. The target audience for this workshop is mainly researchers and practitioners \
working in different fields who are interested in using the Gen-AI tools more \
efficiently and responsibly in their daily work. \
This is a beginner-level workshop. No previous knowledge on the topic is required/expected and the trainer\
will cover the basics of the method. If you want to explore the topic before the workshop \
you can have a look at this Prompt Engineering Guide.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

This text discusses the advancements in Generative AI tools, particularly in the field of Natural Language Processing (NLP), and emphasizes the importance of prompt engineering in order to effectively utilize these tools, with a training workshop aimed at researchers and practitioners interested in using Gen-AI tools efficiently and responsibly.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [8]:
text = f"""
This is a beginner-level workshop. \
No previous knowledge on the topic is required/expected and the trainer \
will cover the basics of the method. If you want to explore the topic \
before the workshop you can have a look at this Prompt Engineering Guide. \
This workshop will be taught by Ozan Evkaya.
"""

prompt = f"""
Generate a list of training details from the given text delimited by triple backticks as instructor, level of workshop and suggested material
Provide them in JSON format with the following keys:
instructor_name, level, material.
"""
response = get_completion(prompt)
print(response)

prompt = f"""
Generate a list of training details from the given ```{text}``` delimited by triple backticks as instructor, level of workshop and suggested material
Provide them in JSON format with the following keys:
instructor_name, level, material.
"""
response = get_completion(prompt)
print(response)

{
  "training_details": [
    {
      "instructor_name": "John Smith",
      "level": "Beginner",
      "material": "Introduction to Python book"
    },
    {
      "instructor_name": "Jane Doe",
      "level": "Intermediate",
      "material": "Python Data Science Handbook"
    },
    {
      "instructor_name": "Michael Johnson",
      "level": "Advanced",
      "material": "Deep Learning with Python"
    }
  ]
}
{
  "instructor_name": "Ozan Evkaya",
  "level": "Beginner",
  "material": "Prompt Engineering Guide"
}


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [9]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Take out the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your cup of tea.


#### Tactic 4: "Few-shot" prompting

In [10]:
prompt = f"""
Your task is to answer in a consistent style.

<tutor>: Teach me about Large Language Models.

<student>: Large language models (LLM) are very large deep learning models \
that are pre-trained on vast amounts of data. \
The underlying transformer is a set of neural networks that \
consist of an encoder and a decoder with self-attention capabilities.

<child>: Teach me about ChatGPT.
"""
response = get_completion(prompt)
print(response)

<assistant>: ChatGPT is a specific large language model developed by OpenAI. It is designed to generate human-like responses in a conversational manner. It has been trained on a wide range of internet text to learn patterns and generate coherent and contextually relevant responses. ChatGPT has been fine-tuned to be more useful and safe for various applications.


### Principle 2: Give the model time to “think”

#### Tactic 1: Specify the steps required to complete a task

Given text below is the abstract of the paper entitled
[Multimodal Large Language Models: A Survey](https://arxiv.org/abs/2311.13165)

In [11]:
text = f"""
The exploration of multimodal language models integrates multiple data types, \
such as images, text, language, audio, and other heterogeneity. \
While the latest large language models excel in text-based tasks, \
they often struggle to understand and process other data types. \
Multimodal models address this limitation by combining various modalities, \
enabling a more comprehensive understanding of diverse data. This paper begins \
by defining the concept of multimodal and examining the historical development of \
multimodal algorithms. Furthermore, we introduce a range of multimodal products, \
focusing on the efforts of major technology companies. A practical guide is provided, \
offering insights into the technical aspects of multimodal models. Moreover, we present a \
compilation of the latest algorithms and commonly used datasets, providing researchers \
with valuable resources for experimentation and evaluation. Lastly, we explore the \
applications of multimodal models and discuss the challenges associated with their development. \
By addressing these aspects, this paper aims to facilitate a deeper understanding of multimodal \
models and their potential in various domains.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into Italian.
3 - Output a json object that contains the following \
keys: summary, italian_summary

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
The exploration of multimodal language models integrates multiple data types, such as images, text, language, audio, and other heterogeneity, to enable a more comprehensive understanding of diverse data, and this paper provides a practical guide, insights into technical aspects, a compilation of algorithms and datasets, and explores the applications and challenges of multimodal models. 

L'esplorazione dei modelli di linguaggio multimodali integra diversi tipi di dati, come immagini, testo, linguaggio, audio e altre eterogeneità, per consentire una comprensione più completa dei dati diversi, e questo articolo fornisce una guida pratica, approfondimenti sugli aspetti tecnici, una raccolta di algoritmi e dataset, ed esplora le applicazioni e le sfide dei modelli multimodali. 

{
  "summary": "The exploration of multimodal language models integrates multiple data types, such as images, text, language, audio, and other heterogeneity, to enable a more comprehensive 

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [12]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations is indeed 450x + 100,000.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [13]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
2. Solar panel cost: $250 / square foot
3. Maintenance cost: $100,000 (flat fee) + $10 / square foot

Let x be the size of the installation in square feet.

1. Land cost: 100 * x
2. Solar panel cost: 250 * x
3. Maintenance cost: 100,000 + 10 * x

Total cost: 100 * x + 250 * x + 100,000 + 10 * x = 350 * x + 100,000

Is the student's solution the same as the actual solution just calculated:
No

Student grade:
Incorrect


## Summarize with a word/sentence/character limit

In [14]:
show_review = """
Welcome to Duffy and Elf Lyons' bank heist. \
A hysterically funny physical show with Visual Vernacular (VV), \
conducted in British Sign Language with some very violent live sound foley. \
Duffy and Elf are friends. Duffy signs, Elf doesn’t. Elf speaks, Duffy doesn’t. \
Let’s see how this works out… This is a comedy show. This is a bank heist. \
This is a love story. You've never seen anything like it. \
Created by award-winning performers Duffy and Elf Lyons. \
'Praise her genius – and go' ***** (Telegraph).\
Edinburgh Comedy Award Best Show nominee.
"""

In [15]:
prompt = f"""
Your task is to generate a short summary of a show description \
shared by Fringe website to give feedback to the \
interested audience.

Summarize the description below, delimited by triple
backticks, in at most 30 words, and focusing on any aspects \
that mention the unique part of the show.

Review: ```{show_review}```
"""

response = get_completion(prompt)
print(response)

Summary: A unique comedy show featuring a bank heist, combining British Sign Language and live sound foley, with a love story twist. Unprecedented and praised by critics.


## Sentiment (positive/negative)

One of the classical tasks of the field of NLP

In [16]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [17]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfied, pleased, grateful, impressed, happy


In [18]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [19]:
prompt = f"""
Translate the following English text to Italian: \
```Hi, I would like to share some experiences about Generative AI```
"""
response = get_completion(prompt)
print(response)

Ciao, mi piacerebbe condividere alcune esperienze riguardo all'IA generativa.


## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [20]:
prompt = f"""
Translate the following from formal to a casual response:
'I regret to inform you that you are not short listed for that specific position'
"""
response = get_completion(prompt)
print(response)

I'm sorry, but you didn't make the cut for that job.


Other examples might be (not limited to);

- Format Conversion

- Spellcheck/Grammar check.

In [21]:
# given the sentiment from the review
# and the original customer message, customize the email
sentiment = "positive"

# review for a blender
review = f"""
Elf and Duffy work so well onstage together. \
They’re both amazing physical comedians and I laughed so so much. \
Kudos to their interpreter as well! Go see this show!
"""

In [22]:
prompt = f"""
You are a Fringe customer service AI assistant.
Your task is to send an email reply to a valued audience.
Given the audience email delimited by ```, \
Generate a reply to thank the audience for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to Fringe audience service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear valued audience,

Thank you for taking the time to share your review with us. We greatly appreciate your kind words and are thrilled to hear that you enjoyed the performance of Elf and Duffy. We agree that they are both amazing physical comedians and it's wonderful to know that you laughed so much during the show. We will make sure to pass on your kudos to their interpreter as well.

We are delighted that you had such a positive experience and we hope to welcome you back to Fringe soon. If you have any further feedback or if there's anything else we can assist you with, please don't hesitate to reach out to our audience service team.

Thank you once again for your support and for being a valued audience member.

Best regards,

AI customer agent
